# Parsbert 

### Libraries

In [2]:
! pip install transformers
! pip install nltk
! pip install hazm
import nltk
nltk.download('punkt')

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
     |████████████████████████████████| 596 kB 35.8 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 316 kB 5.4 MB/s 
     |████████████████████████████████| 233 kB 41.0 MB/s 
     |████████████████████████████████| 1.4 MB 42.2 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=f884086f1d68ef422c254223cd7777d9e56a80e0c0384717b1ecb163686ebc99
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154638 sha256=bb6b0b75ec4788a7ea53969db0ee96ed695

True

In [3]:
! wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P ./data
! wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip -P ./data
! unzip ./data/doc_collection.zip -d ./data
! wget https://raw.githubusercontent.com/sobhe/hazm/master/hazm/data/stopwords.dat -P ./data

--2021-12-27 16:24:29--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50854 (50K) [text/plain]
Saving to: ‘./data/evaluation_IR.yml’

evaluation_IR.yml   100%[===================>]  49.66K  --.-KB/s    in 0.01s   

2021-12-27 16:24:29 (4.97 MB/s) - ‘./data/evaluation_IR.yml’ saved [50854/50854]

--2021-12-27 16:24:29--  https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects

### Read data

set path of data in `PATH` variable

In [4]:
PATH = './data/IR_dataset/'
YAML_PATH = './data/evaluation_IR.yml/'
YAML_PATH = YAML_PATH.rstrip('/')
PATH = PATH.rstrip('/')

### Clean Text

In [5]:
import codecs
import hazm
import re

In [65]:
stopwords = [x.strip() for x in codecs.open('./data/stopwords.dat','r','utf-8').readlines()]
normalizer = hazm.Normalizer()

def remove_spaces_reg():
    # return space patterns
    space_pattern = r"[\xad\ufeff\u200e\u200d\u200b\x7f\u202a\u2003\xa0\u206e\u200c\x9d]"
    return re.compile(space_pattern)

def remove_symbols_reg():
    # return symbol patterns
    symbol_patterns = r"(\d|\"|'ٍ|¬|[؛“،,”‘۔’’‘–]|[|\÷+\]\[\)\(\:\-\?»\=\{}\*«»_…\/ـ]|[۰'ٓ۫'ٔ]|[ٓٔ]|[ًٌﹼ،َُ«ٰ»ٖء])"
    return re.compile(symbol_patterns)

def remeove_arabic(text):
    # remove arabic alphabet
    mapping = {
        u"ۀ" : u"ه",
        u"ة" : u"ت",
        u"ي" : u"ی",
        u"ؤ" : u"و",
        u"إ" : u"ا",
        u"ٹ" : u"ت",
        u"ڈ" : u"د",
        u"ئ" : u"ی",
        u"ﻨ" : u"ن",
        u"ﺠ" : u"ج",
        u"ﻣ" : u"م",
        u"ﷲ" : u"",
        u"ﻳ" : u"ی",
        u"ٻ" : u"ب",
        u"ٱ" : u"ا",
        u"ڵ" : u"ل",
        u"ﭘ" : u"پ",
        u"ﻪ" : u"ه",
        u"ﻳ" : u"ی",
        u"ٻ" : u"ب",
        u"ں" : u"ن",
        u"ٶ" : u"و",
        u"ٲ" : u"ا",
        u"ہ" : u"ه",
        u"ﻩ" : u"ه",
        u"ﻩ" : u"ه",
        u"ك" : u"ک",
        u"ﺆ" : u"و",
        u"أ" : u"ا",
        u"ﺪ" : u"د"
    }

    arabic_keys =  re.compile(r"(" + "|".join(mapping.keys()) + r")")
    return arabic_keys.sub(lambda x: mapping[x.group()], text)


spaces_reg = remove_spaces_reg() 
symbols_reg = remove_symbols_reg()


def clean_text(text, allspace=True, punc=True, stopw=True):
    if allspace:
        text = re.sub(spaces_reg, " ", text)
    if punc:
        text = re.sub(symbols_reg, "", text)

    text = remeove_arabic(text)
    text = normalizer.normalize(text)
    
    if stopw:
        regex = r"\b(?:" + "|".join(map(re.escape, stopwords)) + r")\b"
        text = re.sub(regex, " ", text)
        
    text = re.sub("(\s)+", " ", text)
    text = text.strip()
    return text

store txt files into a list named `doc`

In [119]:
docs = []
for index in range(0, 3258):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
      doc = file_reader.read()
      doc = clean_text(doc, allspace=True, punc=True, stopw=False)
      if doc:
        docs.append(doc)

In [120]:
len(docs)

3258

In [121]:
docs[0]

'برخی از هواداران مصدق یا اعضای جبهه ملی که در زمان نخست وزیری مصدق از جبهه ملی یا از هییت وزیران کنار گذاشته شده یا کنار رفتند پس از جدایی از مصدق به انتقاد از کارنامه وی پرداختند و حتی برای سرنگونی اش تلاش کردند. برخی از این افراد عبارت اند از فضل الله زاهدی نخست وزیر کودتا علی امینی حسین مکی که در آغاز سرباز فداکار وطن نامیده شد ولی در پایان به دلیل مخالفت با مصدق از سوی هواداران جبهه ملی سرباز خطاکار وطن خطاب می\u200cشد مظفر بقایی به دلیل اتهام مشارکت در قتل سرتیپ افشار طوس و سپس اتهام شرکت در کودتای مرداد به دستور فرماندار نظامی تهران بازداشت شد و تا روز مرداد در زندان بود. آیت الله کاشانی در تیر در اعتراض به استعفای مصدق تهدید به حکم جهاد کرد ولی در ماه\u200cهای پایانی ضمن انتقاد شدید از کارنامه دولت از زاهدی پشتیبانی می\u200cکرد. او در اعلامیه\u200cای به عنوان تحریم رفراندوم دولت در هشتم مرداد نوشت ابوالحسن حایری زاده در سال در مجلس ضمن مخالفت با لایحه اختیارات ویژه دولت آن را خطر بازگشت دیکتاتوری ساله دانست و سپس در سال بعد با نوشتن نامه به رییس سازمان ملل متحد مصدق را دیکتاتور

In [72]:
import yaml

In [73]:
evaluation = {}
with open(YAML_PATH, "r") as stream:
    try:
        evaluation = dict(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

### Check GPU Availability

In [74]:
import torch
torch.cuda.is_available()

True

### Implementation

In [41]:
top_k = 100

In [42]:
! pip install -qU sentence-transformers

In [43]:
from sentence_transformers import models, SentenceTransformer, util, InputExample, losses

In [44]:
def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)
    
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [57]:
! pip install stanza

     |████████████████████████████████| 432 kB 5.4 MB/s 
     |████████████████████████████████| 170 kB 45.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=2fd4cb126f80be7041a9f0f426b7396f951e4c44e112407dde1efa3d9a43c8ed
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [161]:
# Load the Sentence-Transformer
model = load_st_model('m3hrdadfi/bert-zwnj-wnli-mean-tokens')

In [78]:
import stanza
stanza.download('fa')
nlp = stanza.Pipeline(lang='fa', processors='tokenize')

2021-12-27 17:07:07 INFO: Downloading default packages for language: fa (Persian)...
2021-12-27 17:07:07 INFO: File exists: /root/stanza_resources/fa/default.zip.
2021-12-27 17:07:10 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-12-27 17:07:10 WARNING: Language fa package default expects mwt, which has been added
2021-12-27 17:07:10 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |

2021-12-27 17:07:10 INFO: Use device: gpu
2021-12-27 17:07:10 INFO: Loading: tokenize
2021-12-27 17:07:10 INFO: Loading: mwt
2021-12-27 17:07:10 INFO: Done loading processors!


#### Fine Tune Model

In [84]:
from tqdm import tqdm

In [122]:
examples = []

for key, value in tqdm(evaluation.items()):
  # Most Relevent Document
  examples.append(InputExample(texts=[docs[value['relevant'][0]], key], label=1))
  # High Similar Documents
  for i in value['similar_high']:
    examples.append(InputExample(texts=[docs[i], key], label=0.9))
  # Medium Similar Documents
  for i in value['similar_med'] :
    examples.append(InputExample(texts=[docs[i], key], label=0.5))
  # Low Similar Documents
  for i in value['similar_low']:
    examples.append(InputExample(texts=[docs[i], key], label=0.2))

100%|██████████| 150/150 [00:00<00:00, 9628.21it/s]


In [124]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

In [162]:
corpus_embeddings = model.encode(docs, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

In [163]:
queries = evaluation.keys()

In [164]:
result = []
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()
    result.append(cos_scores)

In [165]:
#We use torch.topk to find the highest 5 scores
top_results_indices = []
top_results_values = []
for cos_score in result:
  top_results_indices.append(torch.topk(cos_score, k=top_k).indices.numpy())
  top_results_values.append(torch.topk(cos_score, k=top_k).values.numpy())

In [166]:
print("Sentence embeddings:")
print(model.parameters())

Sentence embeddings:
<generator object Module.parameters at 0x7f57a73c7c50>


In [167]:
evaluation['آدولف هیتلر  شکست و مرگ']['relevant'][0]

341

In [168]:
!pip install import-ipynb
import import_ipynb

In [169]:
import utils.evaluation as ev

In [170]:
mrr = []
for i,key in enumerate(evaluation.keys()):
  mrr.append(
      list((top_results_indices[i] == evaluation[key]['relevant'][0]).astype(int))
  )
print(ev.mean_reciprocal_rank(mrr))

0.23766986126868525


In [171]:
print(mrr)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [172]:
import numpy as np

In [174]:
patk = []
for i,key in enumerate(evaluation.keys()):
  patk.append(
      ev.precision_at_k(np.isin(top_results_indices[i], (evaluation[key]['relevant'] + evaluation[key]['similar_high'])).astype(int), 10)
  )
sum(patk)/len(patk)

0.19600000000000004

In [ ]:
patk = []
for i,key in enumerate(evaluation.keys()):
  patk.append(
      ev.precision_at_k(np.isin(top_results_indices[i], (evaluation[key]['relevant'] + evaluation[key]['similar_high'])).astype(int), 10)
  )
sum(patk)/len(patk)

### Pooling Methods

In [ ]:
# Max Pooling - Take the max value over time for every dimension. 
def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
    return torch.mean(token_embeddings, 1)[0]

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)